In [21]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord 
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import time
import os
import json
import matplotlib as mp
import json
from multiprocessing import Process, Manager, Value, Array, managers

# VFDB fasta

In [2]:
setA_nt = SeqIO.to_dict(SeqIO.parse("/home/docker_qiime2_share_D/biodb/VFDB/VFDB_setA_nt.fas",
                                   "fasta"))

# VF info table

In [11]:
test_df = pd.read_csv("/home/docker_qiime2_share_D/all_ec/VFDB/op/11128-vf.tab", 
                      header = None,
                      sep = "\t")
test_df

,0,1,2,3,4,5,6,7,8,9,10
0,VFG038285(gb|WP_005308483),ad6f4ad6-03678,82.353,204,22,225,37,240,3.950000e-45,178.0,96
1,VFG038679(gb|WP_016350055),ad6f4ad6-00250,73.140,242,654,890,651,884,1.440000e-13,75.0,40
2,VFG038857(gb|WP_005314307),ad6f4ad6-00253,79.904,209,500,708,512,720,1.160000e-37,154.0,83
3,VFG000710(gb|AAC38364),ad6f4ad6-03666,99.462,372,19,390,1,372,0.000000e+00,676.0,366
4,VFG000739(gb|AAC38392),ad6f4ad6-03638,94.363,2058,1,2057,1,2057,0.000000e+00,3157.0,1709
...,...,...,...,...,...,...,...,...,...,...,...
320,VFG049917(gb|WP_000508696),ad6f4ad6-00714,75.801,1967,1754,3689,1748,3692,0.000000e+00,946.0,512
321,VFG049917(gb|WP_000508696),ad6f4ad6-00716,75.723,968,2769,3716,3,965,9.910000e-130,462.0,250
322,VFG007097(gb|WP_000138174),ad6f4ad6-03680,74.090,907,235,1133,220,1115,6.160000e-98,355.0,192
323,VFG002356(gb|WP_011816614),ad6f4ad6-02396,75.889,2024,49,2057,49,2057,0.000000e+00,1007.0,545


In [14]:
cols = "qseqid sseqid pident length qstart qend sstart send evalue bitscore score"
cols = cols.split(' ')
cols

['qseqid',
 'sseqid',
 'pident',
 'length',
 'qstart',
 'qend',
 'sstart',
 'send',
 'evalue',
 'bitscore',
 'score']

In [16]:
test_df.columns = cols
test_df

,qseqid,sseqid,pident,length,qstart,qend,sstart,send,evalue,bitscore,score
0,VFG038285(gb|WP_005308483),ad6f4ad6-03678,82.353,204,22,225,37,240,3.950000e-45,178.0,96
1,VFG038679(gb|WP_016350055),ad6f4ad6-00250,73.140,242,654,890,651,884,1.440000e-13,75.0,40
2,VFG038857(gb|WP_005314307),ad6f4ad6-00253,79.904,209,500,708,512,720,1.160000e-37,154.0,83
3,VFG000710(gb|AAC38364),ad6f4ad6-03666,99.462,372,19,390,1,372,0.000000e+00,676.0,366
4,VFG000739(gb|AAC38392),ad6f4ad6-03638,94.363,2058,1,2057,1,2057,0.000000e+00,3157.0,1709
...,...,...,...,...,...,...,...,...,...,...,...
320,VFG049917(gb|WP_000508696),ad6f4ad6-00714,75.801,1967,1754,3689,1748,3692,0.000000e+00,946.0,512
321,VFG049917(gb|WP_000508696),ad6f4ad6-00716,75.723,968,2769,3716,3,965,9.910000e-130,462.0,250
322,VFG007097(gb|WP_000138174),ad6f4ad6-03680,74.090,907,235,1133,220,1115,6.160000e-98,355.0,192
323,VFG002356(gb|WP_011816614),ad6f4ad6-02396,75.889,2024,49,2057,49,2057,0.000000e+00,1007.0,545


In [19]:
d = {}

for x in range(len(cols)):
    d[x] = 'str'
d

{0: 'str',
 1: 'str',
 2: 'str',
 3: 'str',
 4: 'str',
 5: 'str',
 6: 'str',
 7: 'str',
 8: 'str',
 9: 'str',
 10: 'str'}

## on all and 75% similarity, 100% coverage

A “hit” was  considered as any genome sequence that matched the entire length of the query sequence with  >75% identity. As a control to prevent false negatives in the BLAST search, DNA sequencing  reads from each representative UAEC isolate were mapped against a reference sequence  constructed by concatenating all the PUF gene sequences with 100 N’s added to the gene  sequences to separate the concatenated genes using Geneious v6.1.7 (85). Any PUF gene that  was completely covered by the DNA sequencing reads to a depth of 4X was considered a “hit”. 

In [166]:
def get_several_op(m_lst, rt_dict, d_k):
    res_dict = {}
    
    for files in m_lst:# a list of several files
        in_df = pd.read_csv("/home/docker_qiime2_share_D/all_ec/VFDB/op/" + files,
                            header = None,
                            sep = "\t",
                            dtype = {0: 'str',
                                     1: 'str',
                                     2: 'float',
                                     3: 'int',
                                     4: 'int',
                                     5: 'int',
                                     6: 'int',
                                     7: 'int',
                                     8: 'float',
                                     9: 'float',
                                     10: 'int'
                                    }
                           )
        
        res_dict[files.replace('.tab', '')] = {}

        for row in in_df.index:
            if in_df.iloc[row, 2] > 75 and in_df.iloc[row, 3] == len(setA_nt[in_df.iloc[row, 0]].seq):
                res_dict[files.replace('.tab', '')][in_df.iloc[row, 0]] = in_df.iloc[row, 2] 

    rt_dict[d_k] = res_dict

In [167]:
start_time_external = time.time()
print(time.asctime())

if __name__ == "__main__":
    mission_lst = np.array_split(os.listdir("/home/docker_qiime2_share_D/all_ec/VFDB/op"), 36)
    
    manager = Manager()
    return_dict = manager.dict()
    
    jobs = []
    
    for x in range(0, 36):
        p = Process(target = get_several_op,
                   args = (mission_lst[x], return_dict, x)
                   )
        jobs.append(p)
        
        p.start()
        
#     p.join()
    
    for z in jobs:
        z.join()
    
end_time_external = time.time()
print("final time usage " + str(end_time_external - start_time_external))
print(time.asctime())

Sun Aug 28 09:28:29 2022
final time usage 1.0913991928100586
Sun Aug 28 09:28:30 2022


In [168]:
len(return_dict[0])

18

In [169]:
return_dict[0]

{'11128-vf': {'VFG000795(gb|WP_000245816)': 96.774,
  'VFG000796(gb|WP_001053840)': 97.748,
  'VFG000800(gb|WP_000381516)': 84.801,
  'VFG000801(gb|WP_001273445)': 91.288,
  'VFG000804(gb|WP_000098793)': 95.329,
  'VFG000809(gb|WP_000803687)': 93.246,
  'VFG000811(gb|WP_001062956)': 98.131,
  'VFG000812(gb|WP_000622545)': 98.136,
  'VFG000813(gb|WP_001037814)': 97.978,
  'VFG000814(gb|WP_001050992)': 99.435,
  'VFG000816(gb|WP_001302733)': 93.386,
  'VFG000817(gb|WP_000716189)': 98.953,
  'VFG000818(gb|WP_001063681)': 98.684,
  'VFG000819(gb|WP_000723942)': 99.545,
  'VFG000820(gb|WP_000087467)': 99.781,
  'VFG000821(gb|WP_000444180)': 99.034,
  'VFG000822(gb|WP_000605370)': 99.462,
  'VFG000823(gb|WP_000720639)': 99.346,
  'VFG000824(gb|WP_001291694)': 99.711,
  'VFG000825(gb|WP_001002833)': 99.485,
  'VFG000826(gb|WP_000379425)': 99.63,
  'VFG000827(gb|WP_001299599)': 99.847,
  'VFG000828(gb|WP_000780681)': 99.694,
  'VFG000829(gb|WP_000151812)': 99.5,
  'VFG000830(gb|WP_001301632)':

## to df csv

In [170]:
df = {}

for k in return_dict.values():
    df = {**k, **df}

In [171]:
len(df)

637

In [172]:
df['GCA_012659825.1_PDT000668200.2_genomic-vf']

{'VFG000795(gb|WP_000245816)': 100.0,
 'VFG000796(gb|WP_001053840)': 100.0,
 'VFG000797(gb|WP_000228583)': 100.0,
 'VFG000799(gb|WP_000935759)': 100.0,
 'VFG000800(gb|WP_000381516)': 100.0,
 'VFG000802(gb|WP_000953255)': 100.0,
 'VFG000803(gb|WP_000627885)': 100.0,
 'VFG000804(gb|WP_000098793)': 100.0,
 'VFG000805(gb|WP_001301454)': 100.0,
 'VFG000806(gb|WP_000492638)': 100.0,
 'VFG000807(gb|WP_001003065)': 100.0,
 'VFG000808(gb|WP_001302249)': 100.0,
 'VFG000809(gb|WP_000803687)': 100.0,
 'VFG000811(gb|WP_001062956)': 100.0,
 'VFG000812(gb|WP_000622545)': 100.0,
 'VFG000813(gb|WP_001037814)': 100.0,
 'VFG000814(gb|WP_001050992)': 100.0,
 'VFG000815(gb|WP_000386949)': 100.0,
 'VFG000816(gb|WP_001302733)': 100.0,
 'VFG000817(gb|WP_000716189)': 100.0,
 'VFG000818(gb|WP_001063681)': 99.781,
 'VFG000819(gb|WP_000723942)': 100.0,
 'VFG000820(gb|WP_000087467)': 100.0,
 'VFG000823(gb|WP_000720639)': 100.0,
 'VFG000824(gb|WP_001291694)': 100.0,
 'VFG000825(gb|WP_001002833)': 100.0,
 'VFG000826

In [173]:
data_f = pd.DataFrame(df).fillna(0)
data_f

,YN-84-vf,YN-85-vf,YN-86-vf,YN-87-vf,YN-88-vf,YN-89-vf,YN-9-vf,YN-90-vf,YN-91-vf,YN-92-vf,...,GCA_002233035.1_ASM223303v1_genomic-vf,GCA_002244815.1_ASM224481v1_genomic-vf,GCA_002246595.3_ASM224659v3_genomic-vf,GCA_002248225.1_ASM224822v1_genomic-vf,GCA_002268295.1_ASM226829v1_genomic-vf,GCA_002326865.1_ASM232686v1_genomic-vf,GCA_002466975.1_ASM246697v1_genomic-vf,GCA_002467475.1_ASM246747v1_genomic-vf,GCA_002474165.1_ASM247416v1_genomic-vf,GCA_002511125.1_ASM251112v1_genomic-vf
VFG000871(gb|WP_000790574),97.844,98.176,98.010,98.176,98.176,97.844,98.176,98.176,100.000,98.176,...,97.844,97.347,97.844,0.000,98.176,97.678,97.844,0.0,98.176,99.502
VFG000872(gb|WP_000044711),98.827,98.660,98.660,98.660,98.660,99.162,98.660,0.000,99.832,0.000,...,98.325,0.000,98.827,98.827,98.492,97.990,98.995,0.0,98.325,99.497
VFG000875(gb|WP_000066579),98.623,98.485,98.485,98.485,98.485,98.623,98.485,98.898,100.000,98.760,...,98.347,98.485,98.209,98.898,98.485,98.623,98.347,0.0,97.934,99.036
VFG000876(gb|WP_000120946),98.900,98.256,98.256,98.256,98.256,98.786,98.256,98.597,100.000,98.597,...,98.673,98.483,98.294,98.521,98.218,98.294,98.559,0.0,98.180,98.976
VFG000877(gb|WP_001244821),99.058,97.363,97.363,97.363,97.363,98.117,97.363,97.928,100.000,97.928,...,97.363,98.117,97.363,96.610,97.363,98.305,97.363,0.0,97.928,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VFG042796(gb|WP_000575683),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG012436(gb|WP_010904784),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG033773(gb|WP_013009221),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG034882(gb|WP_000535079),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000


In [174]:
cols = list(data_f.columns)
for x in range(len(cols)):
    cols[x] = cols[x].replace('-vf', '')
    
data_f.columns = cols

data_f

,YN-84,YN-85,YN-86,YN-87,YN-88,YN-89,YN-9,YN-90,YN-91,YN-92,...,GCA_002233035.1_ASM223303v1_genomic,GCA_002244815.1_ASM224481v1_genomic,GCA_002246595.3_ASM224659v3_genomic,GCA_002248225.1_ASM224822v1_genomic,GCA_002268295.1_ASM226829v1_genomic,GCA_002326865.1_ASM232686v1_genomic,GCA_002466975.1_ASM246697v1_genomic,GCA_002467475.1_ASM246747v1_genomic,GCA_002474165.1_ASM247416v1_genomic,GCA_002511125.1_ASM251112v1_genomic
VFG000871(gb|WP_000790574),97.844,98.176,98.010,98.176,98.176,97.844,98.176,98.176,100.000,98.176,...,97.844,97.347,97.844,0.000,98.176,97.678,97.844,0.0,98.176,99.502
VFG000872(gb|WP_000044711),98.827,98.660,98.660,98.660,98.660,99.162,98.660,0.000,99.832,0.000,...,98.325,0.000,98.827,98.827,98.492,97.990,98.995,0.0,98.325,99.497
VFG000875(gb|WP_000066579),98.623,98.485,98.485,98.485,98.485,98.623,98.485,98.898,100.000,98.760,...,98.347,98.485,98.209,98.898,98.485,98.623,98.347,0.0,97.934,99.036
VFG000876(gb|WP_000120946),98.900,98.256,98.256,98.256,98.256,98.786,98.256,98.597,100.000,98.597,...,98.673,98.483,98.294,98.521,98.218,98.294,98.559,0.0,98.180,98.976
VFG000877(gb|WP_001244821),99.058,97.363,97.363,97.363,97.363,98.117,97.363,97.928,100.000,97.928,...,97.363,98.117,97.363,96.610,97.363,98.305,97.363,0.0,97.928,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VFG042796(gb|WP_000575683),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG012436(gb|WP_010904784),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG033773(gb|WP_013009221),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG034882(gb|WP_000535079),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000


In [175]:
'YN-1' in data_f.columns

True

In [176]:
'YN-59' in data_f.columns

False

# filter 91 with bowtie coverage

In [177]:
test_op_coverage = pd.read_csv("/home/docker_qiime2_share_D/91_EC/VFDB/bw/op_cov_all/A_1-cov.txt",
                               sep = "\t",
                               index_col = 0)
test_op_coverage = test_op_coverage.sort_values(by = ["numreads"], 
                                                ascending = False)
test_op_coverage

,startpos,endpos,numreads,covbases,coverage,meandepth,meanbaseq,meanmapq
#rname,,,,,,,,
VFG049148(gb|WP_041168941),1,9621,22379,9621,100.0,350.783,36.3,255.00
VFG000362(gb|WP_002212777),1,9492,16472,9492,100.0,261.760,36.4,4.25
VFG049157(gb|WP_014907259),1,6465,15737,6465,100.0,367.040,36.3,206.00
VFG049156(gb|WP_077265119),1,6432,14022,6432,100.0,328.632,36.2,223.00
VFG049154(gb|WP_014907257),1,4797,11701,4797,100.0,367.766,36.3,255.00
...,...,...,...,...,...,...,...,...
VFG046187(gb|NP_820476),1,1674,0,0,0.0,0.000,0.0,0.00
VFG046191,1,2368,0,0,0.0,0.000,0.0,0.00
VFG046194(gb|NP_820513),1,1938,0,0,0.0,0.000,0.0,0.00


In [178]:
test_op_coverage.loc['VFG000861(gb|WP_001045649)']

startpos        1.00000
endpos       4119.00000
numreads        7.00000
covbases      794.00000
coverage       19.27650
meandepth       0.25613
meanbaseq      36.60000
meanmapq        1.00000
Name: VFG000861(gb|WP_001045649), dtype: float64

In [179]:
data_f.loc['VFG000861(gb|WP_001045649)', 'YN-1']

97.014

In [180]:
True | False and False

False

In [181]:
count = 0

index = list(data_f.index)

for bw_cov in os.listdir('/home/docker_qiime2_share_D/91_EC/VFDB/bw/op_cov_all'):
#     print(bw_cov)
    coverage = pd.read_csv("/home/docker_qiime2_share_D/91_EC/VFDB/bw/op_cov_all/" + bw_cov,
                           sep = "\t",
                           index_col = 0)
    
    for vfs in coverage.index:
        if vfs not in index:
            continue
        else:
            cov = coverage.loc[vfs, 'coverage'] < 100
            depth = coverage.loc[vfs, 'meandepth'] < 4 
            hit_by_blast = data_f.loc[vfs, bw_cov.replace('-cov.txt', '').replace('A_', 'YN-')] > 0
            
            if cov | depth and hit_by_blast:
                data_f.loc[vfs, bw_cov.replace('-cov.txt', '').replace('A_', 'YN-')] = 0
                count += 1
                print(bw_cov.replace('-cov.txt', '').replace('A_', 'YN-') + " has no " + vfs)

YN-1 has no VFG000861(gb|WP_001045649)
YN-1 has no VFG000895(gb|WP_000654937)
YN-1 has no VFG000938(gb|WP_001015709)
YN-1 has no VFG000939(gb|WP_001287497)
YN-1 has no VFG000940(gb|WP_001299620)
YN-1 has no VFG012329(gb|WP_000046976)
YN-1 has no VFG012335(gb|WP_000975448)
YN-1 has no VFG012591(gb|WP_001017167)
YN-1 has no VFG012597(gb|WP_000089593)
YN-1 has no VFG012603(gb|WP_001302516)
YN-1 has no VFG012609(gb|WP_000993317)
YN-1 has no VFG012615(gb|WP_000020049)
YN-1 has no VFG012621(gb|WP_000189352)
YN-1 has no VFG033389(gb|WP_000237766)
YN-1 has no VFG044323(gb|WP_015874960)
YN-1 has no VFG044324(gb|WP_032447693)
YN-1 has no VFG044325(gb|WP_002212761)
YN-1 has no VFG044326(gb|WP_001327262)
YN-1 has no VFG044327(gb|WP_000140406)
YN-1 has no VFG044328(gb|WP_015874963)
YN-1 has no VFG044329(gb|WP_015874964)
YN-1 has no VFG044330(gb|WP_032447695)
YN-1 has no VFG044331(gb|WP_015874966)
YN-1 has no VFG044332(gb|WP_015874967)
YN-1 has no VFG044333(gb|WP_015874968)
YN-1 has no VFG048603(gb|

YN-110 has no VFG000916(gb|WP_001017192)
YN-110 has no VFG000919(gb|WP_001095619)
YN-110 has no VFG000920(gb|WP_000020034)
YN-110 has no VFG000929(gb|WP_001375021)
YN-110 has no VFG012591(gb|WP_001017167)
YN-110 has no VFG012597(gb|WP_000089593)
YN-110 has no VFG012603(gb|WP_001302516)
YN-110 has no VFG012609(gb|WP_000993317)
YN-110 has no VFG012615(gb|WP_000020049)
YN-110 has no VFG034624(gb|WP_000755097)
YN-110 has no VFG034754(gb|WP_000081129)
YN-110 has no VFG034849(gb|WP_000129338)
YN-110 has no VFG034851(gb|WP_000129342)
YN-110 has no VFG034866(gb|WP_000900547)
YN-110 has no VFG034895(gb|WP_001304001)
YN-110 has no VFG034897(gb|WP_013009101)
YN-110 has no VFG034901(gb|WP_000868321)
YN-110 has no VFG044172(gb|WP_000622316)
YN-110 has no VFG044323(gb|WP_015874960)
YN-110 has no VFG044324(gb|WP_032447693)
YN-110 has no VFG044325(gb|WP_002212761)
YN-110 has no VFG044326(gb|WP_001327262)
YN-110 has no VFG044327(gb|WP_000140406)
YN-110 has no VFG044328(gb|WP_015874963)
YN-110 has no VF

YN-2 has no VFG000939(gb|WP_001287497)
YN-2 has no VFG000940(gb|WP_001299620)
YN-2 has no VFG012591(gb|WP_001017167)
YN-2 has no VFG012597(gb|WP_000089593)
YN-2 has no VFG012609(gb|WP_000993317)
YN-2 has no VFG044323(gb|WP_015874960)
YN-2 has no VFG044324(gb|WP_032447693)
YN-2 has no VFG044325(gb|WP_002212761)
YN-2 has no VFG044326(gb|WP_001327262)
YN-2 has no VFG044327(gb|WP_000140406)
YN-2 has no VFG044328(gb|WP_015874963)
YN-2 has no VFG044329(gb|WP_015874964)
YN-2 has no VFG044330(gb|WP_032447695)
YN-2 has no VFG044331(gb|WP_015874966)
YN-2 has no VFG044332(gb|WP_015874967)
YN-2 has no VFG048603(gb|WP_004213921)
YN-2 has no VFG049018(gb|WP_002913007)
YN-2 has no VFG000617(gb|NP_709456)
YN-2 has no VFG013063(gb|WP_001017208)
YN-2 has no VFG013064(gb|WP_000089574)
YN-2 has no VFG013065(gb|WP_024259428)
YN-2 has no VFG013066
YN-2 has no VFG013069(gb|WP_005019013)
YN-2 has no VFG013070(gb|WP_001626196)
YN-2 has no VFG000365(gb|WP_002214412)
YN-2 has no VFG012603(gb|WP_001302516)
YN-2 h

YN-31 has no VFG048603(gb|WP_004213921)
YN-32 has no VFG000881(gb|WP_000930062)
YN-32 has no VFG000883(gb|WP_001239363)
YN-32 has no VFG000894(gb|WP_001239363)
YN-32 has no VFG034849(gb|WP_000129338)
YN-32 has no VFG034873(gb|WP_001270084)
YN-32 has no VFG045793(gb|WP_000771429)
YN-32 has no VFG048603(gb|WP_004213921)
YN-32 has no VFG048980(gb|WP_041937790)
YN-33 has no VFG000859(gb|CBG37421)
YN-33 has no VFG000860(gb|CBG37420)
YN-33 has no VFG000916(gb|WP_001017192)
YN-33 has no VFG000917(gb|WP_000089583)
YN-33 has no VFG000918(gb|WP_001312167)
YN-33 has no VFG000919(gb|WP_001095619)
YN-33 has no VFG000920(gb|WP_000020034)
YN-33 has no VFG000921(gb|WP_000189356)
YN-33 has no VFG012597(gb|WP_000089593)
YN-33 has no VFG034849(gb|WP_000129338)
YN-33 has no VFG034851(gb|WP_000129342)
YN-33 has no VFG034873(gb|WP_001270084)
YN-33 has no VFG034908(gb|WP_000978349)
YN-33 has no VFG044172(gb|WP_000622316)
YN-33 has no VFG000636(gb|AAW31739)
YN-33 has no VFG000637(gb|AAW31740)
YN-33 has no VFG

YN-41 has no VFG000916(gb|WP_001017192)
YN-41 has no VFG000919(gb|WP_001095619)
YN-41 has no VFG000920(gb|WP_000020034)
YN-41 has no VFG000921(gb|WP_000189356)
YN-41 has no VFG000929(gb|WP_001375021)
YN-41 has no VFG000940(gb|WP_001299620)
YN-41 has no VFG012591(gb|WP_001017167)
YN-41 has no VFG012597(gb|WP_000089593)
YN-41 has no VFG012603(gb|WP_001302516)
YN-41 has no VFG012609(gb|WP_000993317)
YN-41 has no VFG012615(gb|WP_000020049)
YN-41 has no VFG034624(gb|WP_000755097)
YN-41 has no VFG034754(gb|WP_000081129)
YN-41 has no VFG034866(gb|WP_000900547)
YN-41 has no VFG044172(gb|WP_000622316)
YN-41 has no VFG044323(gb|WP_015874960)
YN-41 has no VFG044324(gb|WP_032447693)
YN-41 has no VFG044325(gb|WP_002212761)
YN-41 has no VFG044326(gb|WP_001327262)
YN-41 has no VFG044327(gb|WP_000140406)
YN-41 has no VFG044328(gb|WP_015874963)
YN-41 has no VFG044331(gb|WP_015874966)
YN-41 has no VFG044332(gb|WP_015874967)
YN-41 has no VFG044333(gb|WP_015874968)
YN-41 has no VFG048603(gb|WP_004213921)


YN-52 has no VFG000928(gb|WP_000640938)
YN-52 has no VFG044323(gb|WP_015874960)
YN-52 has no VFG044324(gb|WP_032447693)
YN-52 has no VFG044325(gb|WP_002212761)
YN-52 has no VFG044326(gb|WP_001327262)
YN-52 has no VFG044327(gb|WP_000140406)
YN-52 has no VFG044328(gb|WP_015874963)
YN-52 has no VFG044329(gb|WP_015874964)
YN-52 has no VFG044330(gb|WP_032447695)
YN-52 has no VFG044331(gb|WP_015874966)
YN-52 has no VFG044332(gb|WP_015874967)
YN-52 has no VFG044333(gb|WP_015874968)
YN-52 has no VFG002055(gb|WP_000097229)
YN-52 has no VFG000365(gb|WP_002214412)
YN-53 has no VFG000859(gb|CBG37421)
YN-53 has no VFG000861(gb|WP_001045649)
YN-53 has no VFG000895(gb|WP_000654937)
YN-53 has no VFG000938(gb|WP_001015709)
YN-53 has no VFG000939(gb|WP_001287497)
YN-53 has no VFG000940(gb|WP_001299620)
YN-53 has no VFG001447(gb|CAA50007)
YN-53 has no VFG012329(gb|WP_000046976)
YN-53 has no VFG012335(gb|WP_000975448)
YN-53 has no VFG012591(gb|WP_001017167)
YN-53 has no VFG012597(gb|WP_000089593)
YN-53 ha

YN-63 has no VFG000928(gb|WP_000640938)
YN-63 has no VFG034873(gb|WP_001270084)
YN-63 has no VFG044323(gb|WP_015874960)
YN-63 has no VFG044324(gb|WP_032447693)
YN-63 has no VFG044325(gb|WP_002212761)
YN-63 has no VFG044326(gb|WP_001327262)
YN-63 has no VFG044327(gb|WP_000140406)
YN-63 has no VFG044328(gb|WP_015874963)
YN-63 has no VFG044329(gb|WP_015874964)
YN-63 has no VFG044330(gb|WP_032447695)
YN-63 has no VFG044331(gb|WP_015874966)
YN-63 has no VFG044332(gb|WP_015874967)
YN-63 has no VFG044333(gb|WP_015874968)
YN-63 has no VFG002052(gb|WP_000820136)
YN-63 has no VFG000364(gb|WP_000140406)
YN-63 has no VFG000365(gb|WP_002214412)
YN-65 has no VFG000861(gb|WP_001045649)
YN-65 has no VFG000895(gb|WP_000654937)
YN-65 has no VFG000911(gb|WP_000975445)
YN-65 has no VFG000912(gb|WP_001443207)
YN-65 has no VFG001447(gb|CAA50007)
YN-65 has no VFG012329(gb|WP_000046976)
YN-65 has no VFG012337(gb|WP_001370792)
YN-65 has no VFG012591(gb|WP_001017167)
YN-65 has no VFG012597(gb|WP_000089593)
YN-6

YN-73 has no VFG034873(gb|WP_001270084)
YN-73 has no VFG044323(gb|WP_015874960)
YN-73 has no VFG044324(gb|WP_032447693)
YN-73 has no VFG044325(gb|WP_002212761)
YN-73 has no VFG044326(gb|WP_001327262)
YN-73 has no VFG044327(gb|WP_000140406)
YN-73 has no VFG044328(gb|WP_015874963)
YN-73 has no VFG044329(gb|WP_015874964)
YN-73 has no VFG044330(gb|WP_032447695)
YN-73 has no VFG044331(gb|WP_015874966)
YN-73 has no VFG044332(gb|WP_015874967)
YN-73 has no VFG044333(gb|WP_015874968)
YN-73 has no VFG048550(gb|WP_004213623)
YN-73 has no VFG048603(gb|WP_004213921)
YN-73 has no VFG000365(gb|WP_002214412)
YN-73 has no VFG044335(gb|WP_015874980)
YN-74 has no VFG000895(gb|WP_000654937)
YN-74 has no VFG000938(gb|WP_001015709)
YN-74 has no VFG000939(gb|WP_001287497)
YN-74 has no VFG000940(gb|WP_001299620)
YN-74 has no VFG001447(gb|CAA50007)
YN-74 has no VFG012591(gb|WP_001017167)
YN-74 has no VFG012597(gb|WP_000089593)
YN-74 has no VFG012609(gb|WP_000993317)
YN-74 has no VFG012621(gb|WP_000189352)
YN-7

YN-82 has no VFG000939(gb|WP_001287497)
YN-82 has no VFG000940(gb|WP_001299620)
YN-82 has no VFG012591(gb|WP_001017167)
YN-82 has no VFG012597(gb|WP_000089593)
YN-82 has no VFG012603(gb|WP_001302516)
YN-82 has no VFG012609(gb|WP_000993317)
YN-82 has no VFG012621(gb|WP_000189352)
YN-82 has no VFG044323(gb|WP_015874960)
YN-82 has no VFG044324(gb|WP_032447693)
YN-82 has no VFG044325(gb|WP_002212761)
YN-82 has no VFG044326(gb|WP_001327262)
YN-82 has no VFG044327(gb|WP_000140406)
YN-82 has no VFG044328(gb|WP_015874963)
YN-82 has no VFG044329(gb|WP_015874964)
YN-82 has no VFG044330(gb|WP_032447695)
YN-82 has no VFG044331(gb|WP_015874966)
YN-82 has no VFG044332(gb|WP_015874967)
YN-82 has no VFG044333(gb|WP_015874968)
YN-82 has no VFG048603(gb|WP_004213921)
YN-82 has no VFG049018(gb|WP_002913007)
YN-82 has no VFG000617(gb|NP_709456)
YN-82 has no VFG013063(gb|WP_001017208)
YN-82 has no VFG013064(gb|WP_000089574)
YN-82 has no VFG013065(gb|WP_024259428)
YN-82 has no VFG013066
YN-82 has no VFG0130

YN-91 has no VFG000636(gb|AAW31739)
YN-91 has no VFG000637(gb|AAW31740)
YN-91 has no VFG013063(gb|WP_001017208)
YN-91 has no VFG013066
YN-92 has no VFG000928(gb|WP_000640938)
YN-92 has no VFG034849(gb|WP_000129338)
YN-92 has no VFG034873(gb|WP_001270084)
YN-92 has no VFG045793(gb|WP_000771429)
YN-92 has no VFG044323(gb|WP_015874960)
YN-92 has no VFG044324(gb|WP_032447693)
YN-92 has no VFG044325(gb|WP_002212761)
YN-92 has no VFG044326(gb|WP_001327262)
YN-92 has no VFG044327(gb|WP_000140406)
YN-92 has no VFG044328(gb|WP_015874963)
YN-92 has no VFG044329(gb|WP_015874964)
YN-92 has no VFG044330(gb|WP_032447695)
YN-92 has no VFG044331(gb|WP_015874966)
YN-92 has no VFG044332(gb|WP_015874967)
YN-92 has no VFG044333(gb|WP_015874968)
YN-92 has no VFG048603(gb|WP_004213921)
YN-92 has no VFG048980(gb|WP_041937790)
YN-92 has no VFG000616(gb|NP_709455)
YN-92 has no VFG000617(gb|NP_709456)
YN-92 has no VFG000365(gb|WP_002214412)
YN-93 has no VFG000873(gb|WP_000695571)
YN-93 has no VFG000916(gb|WP_00

In [182]:
print(count)

2546


In [183]:
data_f

,YN-84,YN-85,YN-86,YN-87,YN-88,YN-89,YN-9,YN-90,YN-91,YN-92,...,GCA_002233035.1_ASM223303v1_genomic,GCA_002244815.1_ASM224481v1_genomic,GCA_002246595.3_ASM224659v3_genomic,GCA_002248225.1_ASM224822v1_genomic,GCA_002268295.1_ASM226829v1_genomic,GCA_002326865.1_ASM232686v1_genomic,GCA_002466975.1_ASM246697v1_genomic,GCA_002467475.1_ASM246747v1_genomic,GCA_002474165.1_ASM247416v1_genomic,GCA_002511125.1_ASM251112v1_genomic
VFG000871(gb|WP_000790574),97.844,98.176,98.010,98.176,98.176,97.844,98.176,98.176,100.000,98.176,...,97.844,97.347,97.844,0.000,98.176,97.678,97.844,0.0,98.176,99.502
VFG000872(gb|WP_000044711),98.827,98.660,98.660,98.660,98.660,99.162,98.660,0.000,99.832,0.000,...,98.325,0.000,98.827,98.827,98.492,97.990,98.995,0.0,98.325,99.497
VFG000875(gb|WP_000066579),98.623,98.485,98.485,98.485,98.485,98.623,98.485,98.898,100.000,98.760,...,98.347,98.485,98.209,98.898,98.485,98.623,98.347,0.0,97.934,99.036
VFG000876(gb|WP_000120946),98.900,98.256,98.256,98.256,98.256,98.786,98.256,98.597,100.000,98.597,...,98.673,98.483,98.294,98.521,98.218,98.294,98.559,0.0,98.180,98.976
VFG000877(gb|WP_001244821),99.058,97.363,97.363,97.363,97.363,98.117,97.363,97.928,100.000,97.928,...,97.363,98.117,97.363,96.610,97.363,98.305,97.363,0.0,97.928,100.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VFG042796(gb|WP_000575683),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG012436(gb|WP_010904784),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG033773(gb|WP_013009221),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000
VFG034882(gb|WP_000535079),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000


In [185]:
os.getcwd()

'/home/docker_qiime2_share_D/all_ec/VFDB'

In [184]:
data_f.to_csv('cov_filtered_vf.csv')